In [1]:
import cx_Oracle as ora
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [2]:
def setKoreanFont() :
    from matplotlib import font_manager, rc
    font_name = font_manager.FontProperties(fname="C:/Windows/Fonts/gulim.ttc").get_name()
    rc("font", family=font_name)
    plt.rcParams["font.size"]=15
    plt.rcParams["figure.figsize"] = 14,8

In [3]:
# Oracle 연결 : 
con = ora.connect("WJV/wjvc3005@10.101.1.212:1521/CDW")

df_rslt = pd.read_sql("""

SELECT 
       AVG_PORD_INCR_PCNT*100 AS AVG_PORD_INCR_PCNT,
       PPP AS PPP,
       ROUND(AVG_CNCL_CNT)/10000 AVG_CNCL_CNT,
       OWNE_CNT/10000 AS OWNE_CNT,
       HG_PCNT*100 AS HG_PCNT,
       PORD_PCNT*10 AS PORD_PCNT,
       PORD_CNT
  FROM SULIM_INDEX_PRDC_PORD_CNT

""", con=con)

con.close()
print(df_rslt)

     AVG_PORD_INCR_PCNT     PPP  AVG_CNCL_CNT  OWNE_CNT  HG_PCNT  PORD_PCNT  \
0                -5.266   8.169        5.6100    4.4996    9.579     9.7045   
1                 7.232   8.818        4.4527    5.2277    5.615     9.8042   
2                 1.895   8.526        4.6737    4.3690    3.467     9.3592   
3                 4.543   9.270        3.2115    3.9586    4.246     8.2428   
4                 3.758   7.620        4.6474    3.8275    2.217     9.7021   
5                -4.930   7.345        3.4136    4.2877    4.726     8.4069   
6                 0.560   7.721        4.0790    5.3028    4.113     9.7557   
7                -5.534   8.941        5.5622    3.9908   13.533     9.8742   
8                 7.380   7.751        3.7979    3.6755    4.067     9.5962   
9                 7.402   8.616        4.5624    3.5098    4.233     9.2665   
10                0.182   9.193        4.2204    4.4465    4.755     9.5452   
11                7.888   8.524        4.4077    4.2

In [4]:
mode = tf.global_variables_initializer()

In [5]:
xy = np.array(df_rslt, dtype=np.float32)
# print(xy)

In [6]:
# 다변인 선형회귀 : 모델에 영향을 미치는 변인이 여러개 일때
# 가격정보제외하고 가져오기
x_data = xy[:,0:-1]
print(x_data)
# 가격정보만 가저오기
y_data = xy[:,[-1]]
# print(y_data)

[[-5.26600e+00  8.16900e+00  5.61000e+00  4.49960e+00  9.57900e+00
   9.70450e+00]
 [ 7.23200e+00  8.81800e+00  4.45270e+00  5.22770e+00  5.61500e+00
   9.80420e+00]
 [ 1.89500e+00  8.52600e+00  4.67370e+00  4.36900e+00  3.46700e+00
   9.35920e+00]
 [ 4.54300e+00  9.27000e+00  3.21150e+00  3.95860e+00  4.24600e+00
   8.24280e+00]
 [ 3.75800e+00  7.62000e+00  4.64740e+00  3.82750e+00  2.21700e+00
   9.70210e+00]
 [-4.93000e+00  7.34500e+00  3.41360e+00  4.28770e+00  4.72600e+00
   8.40690e+00]
 [ 5.60000e-01  7.72100e+00  4.07900e+00  5.30280e+00  4.11300e+00
   9.75570e+00]
 [-5.53400e+00  8.94100e+00  5.56220e+00  3.99080e+00  1.35330e+01
   9.87420e+00]
 [ 7.38000e+00  7.75100e+00  3.79790e+00  3.67550e+00  4.06700e+00
   9.59620e+00]
 [ 7.40200e+00  8.61600e+00  4.56240e+00  3.50980e+00  4.23300e+00
   9.26650e+00]
 [ 1.82000e-01  9.19300e+00  4.22040e+00  4.44650e+00  4.75500e+00
   9.54520e+00]
 [ 7.88800e+00  8.52400e+00  4.40770e+00  4.26450e+00  5.86800e+00
   9.69170e+00]
 [ 2

In [7]:
# placeholder 만들기
X = tf.placeholder(tf.float32, shape=[None,6])
Y = tf.placeholder(tf.float32, shape=[None,1])
# 가중치값 초기화
W = tf.Variable(tf.random_normal([6,1],name='weight'))
# 바이어스값 초기화
b = tf.Variable(tf.random_normal([1]), name='bias')

In [8]:
# 가설식 설정
hypothesis = tf.matmul(X,W)+b
# 비용함수 설정
cost = tf.reduce_mean(tf.square(hypothesis-Y))
# 최적화함수 설정
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.000005)
train = optimizer.minimize(cost)

In [9]:
# 세션 초기화
sess = tf.Session()
# 글로벌 변수 초기화
sess.run(tf.global_variables_initializer())
# 학습진행
for step in range(300001) :
    cost_, hypo_, _ = sess.run([cost,hypothesis,train],feed_dict={X:x_data,Y:y_data})
    if step % 500 == 0:
        print("#",step," 손실 비용:",cost_)
        print("- 판매량 : ", hypo_[0])

# 0  손실 비용: 15423372000.0
- 판매량 :  [1.6898006]
# 500  손실 비용: 1715541400.0
- 판매량 :  [92134.59]
# 1000  손실 비용: 390151680.0
- 판매량 :  [120927.71]
# 1500  손실 비용: 260137900.0
- 판매량 :  [130000.81]
# 2000  손실 비용: 245833220.0
- 판매량 :  [132912.2]
# 2500  손실 비용: 242958660.0
- 판매량 :  [133880.31]
# 3000  손실 비용: 241387230.0
- 판매량 :  [134221.12]
# 3500  손실 비용: 240096600.0
- 판매량 :  [134348.12]
# 4000  손실 비용: 238955790.0
- 판매량 :  [134393.34]
# 4500  손실 비용: 237928670.0
- 판매량 :  [134400.89]
# 5000  손실 비용: 236995070.0
- 판매량 :  [134387.16]
# 5500  손실 비용: 236140530.0
- 판매량 :  [134359.]
# 6000  손실 비용: 235353810.0
- 판매량 :  [134320.69]
# 6500  손실 비용: 234626560.0
- 판매량 :  [134274.94]
# 7000  손실 비용: 233951980.0
- 판매량 :  [134222.9]
# 7500  손실 비용: 233324850.0
- 판매량 :  [134166.8]
# 8000  손실 비용: 232740420.0
- 판매량 :  [134107.81]
# 8500  손실 비용: 232194990.0
- 판매량 :  [134046.81]
# 9000  손실 비용: 231685120.0
- 판매량 :  [133984.7]
# 9500  손실 비용: 231208200.0
- 판매량 :  [133921.86]
# 10000  손실 비용: 230761380.0
- 판매량 :  [133859.02]

# 84500  손실 비용: 219554690.0
- 판매량 :  [132048.73]
# 85000  손실 비용: 219530260.0
- 판매량 :  [132049.67]
# 85500  손실 비용: 219506140.0
- 판매량 :  [132050.58]
# 86000  손실 비용: 219482220.0
- 판매량 :  [132051.48]
# 86500  손실 비용: 219458380.0
- 판매량 :  [132052.39]
# 87000  손실 비용: 219434640.0
- 판매량 :  [132053.28]
# 87500  손실 비용: 219411120.0
- 판매량 :  [132054.14]
# 88000  손실 비용: 219387950.0
- 판매량 :  [132055.02]
# 88500  손실 비용: 219364700.0
- 판매량 :  [132055.77]
# 89000  손실 비용: 219342000.0
- 판매량 :  [132056.12]
# 89500  손실 비용: 219319400.0
- 판매량 :  [132056.47]
# 90000  손실 비용: 219296830.0
- 판매량 :  [132057.16]
# 90500  손실 비용: 219274380.0
- 판매량 :  [132058.1]
# 91000  손실 비용: 219251980.0
- 판매량 :  [132059.03]
# 91500  손실 비용: 219229700.0
- 판매량 :  [132059.97]
# 92000  손실 비용: 219207460.0
- 판매량 :  [132060.9]
# 92500  손실 비용: 219185300.0
- 판매량 :  [132061.81]
# 93000  손실 비용: 219163200.0
- 판매량 :  [132062.72]
# 93500  손실 비용: 219141220.0
- 판매량 :  [132063.64]
# 94000  손실 비용: 219119730.0
- 판매량 :  [132063.94]
# 94500  손실 비용: 219098

- 판매량 :  [132184.62]
# 168000  손실 비용: 217081260.0
- 판매량 :  [132185.33]
# 168500  손실 비용: 217073220.0
- 판매량 :  [132186.02]
# 169000  손실 비용: 217065170.0
- 판매량 :  [132186.69]
# 169500  손실 비용: 217057150.0
- 판매량 :  [132187.36]
# 170000  손실 비용: 217049150.0
- 판매량 :  [132188.06]
# 170500  손실 비용: 217041200.0
- 판매량 :  [132188.73]
# 171000  손실 비용: 217033220.0
- 판매량 :  [132189.4]
# 171500  손실 비용: 217025280.0
- 판매량 :  [132190.11]
# 172000  손실 비용: 217017360.0
- 판매량 :  [132190.8]
# 172500  손실 비용: 217009550.0
- 판매량 :  [132191.47]
# 173000  손실 비용: 217001680.0
- 판매량 :  [132192.16]
# 173500  손실 비용: 216993820.0
- 판매량 :  [132192.84]
# 174000  손실 비용: 216986100.0
- 판매량 :  [132193.36]
# 174500  손실 비용: 216978720.0
- 판매량 :  [132193.34]
# 175000  손실 비용: 216971400.0
- 판매량 :  [132193.67]
# 175500  손실 비용: 216964000.0
- 판매량 :  [132194.]
# 176000  손실 비용: 216956690.0
- 판매량 :  [132194.31]
# 176500  손실 비용: 216949390.0
- 판매량 :  [132194.66]
# 177000  손실 비용: 216942160.0
- 판매량 :  [132194.98]
# 177500  손실 비용: 216934880.0
- 판매

# 250500  손실 비용: 216227740.0
- 판매량 :  [132279.12]
# 251000  손실 비용: 216224750.0
- 판매량 :  [132279.73]
# 251500  손실 비용: 216221740.0
- 판매량 :  [132280.34]
# 252000  손실 비용: 216218770.0
- 판매량 :  [132280.81]
# 252500  손실 비용: 216215780.0
- 판매량 :  [132281.19]
# 253000  손실 비용: 216212830.0
- 판매량 :  [132281.6]
# 253500  손실 비용: 216209870.0
- 판매량 :  [132281.98]
# 254000  손실 비용: 216206900.0
- 판매량 :  [132282.36]
# 254500  손실 비용: 216203950.0
- 판매량 :  [132282.77]
# 255000  손실 비용: 216201040.0
- 판매량 :  [132283.14]
# 255500  손실 비용: 216198080.0
- 판매량 :  [132283.53]
# 256000  손실 비용: 216195180.0
- 판매량 :  [132283.94]
# 256500  손실 비용: 216192270.0
- 판매량 :  [132284.34]
# 257000  손실 비용: 216189330.0
- 판매량 :  [132284.77]
# 257500  손실 비용: 216186480.0
- 판매량 :  [132285.14]
# 258000  손실 비용: 216183580.0
- 판매량 :  [132285.55]
# 258500  손실 비용: 216180720.0
- 판매량 :  [132285.95]
# 259000  손실 비용: 216177840.0
- 판매량 :  [132286.38]
# 259500  손실 비용: 216174980.0
- 판매량 :  [132286.77]
# 260000  손실 비용: 216172100.0
- 판매량 :  [132287.17]
#

In [10]:
# 학습모델 저장 (간단한방법)
saver = tf.train.Saver()
save_path = saver.save(sess, "./COP/saved.cpkt")
print("학습된 모델을 저장했습니다.")

학습된 모델을 저장했습니다.


In [236]:
rslt = sess.run(hypothesis,feed_dict={X:((7.888,8.524,4.4077,4.2645,5.868,9.6917),)})
print(rslt, rslt/137089)
# 7월예상 : 140535.58

[[127382.37]] [[0.92919466]]
